In [1]:
import pickle
import pandas as pd

# importing Libraries
from tqdm import tqdm
import shutil
import os 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import random
from wordcloud import WordCloud

# Data preprtocessing

import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import Normalizer


# Model
from sklearn.model_selection import train_test_split


from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier,StackingClassifier,VotingClassifier

# matrics

from sklearn.metrics import log_loss,accuracy_score,confusion_matrix,f1_score


# libraries for deep learning model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model



In [77]:
!pip install gensim


Defaulting to user installation because normal site-packages is not writeable


In [78]:
pip install fasttext


Defaulting to user installation because normal site-packages is not writeable
  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Running setup.py clean for fasttext
Failed to build fasttext
    Running setup.py install for fasttext: started
    Running setup.py install for fasttext: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Windows\\Temp\\pip-install-6nysi9mb\\fasttext_d01c9b9ede194e308740365123c10cd3\\setup.py'"'"'; __file__='"'"'C:\\Windows\\Temp\\pip-install-6nysi9mb\\fasttext_d01c9b9ede194e308740365123c10cd3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Windows\Temp\pip-wheel-94bq030p'
       cwd: C:\Windows\Temp\pip-install-6nysi9mb\fasttext_d01c9b9ede194e308740365123c10cd3\
  Complete output (20 lines):
  C:\ProgramData\Anaconda3\lib\site-packages\setuptools\dist.py:757: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the undersc

In [2]:
pip install fasttext

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Running setup.py clean for fasttext
Failed to build fasttext
    Running setup.py install for fasttext: started
    Running setup.py install for fasttext: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Windows\\Temp\\pip-install-5z_34s41\\fasttext_bcbff0f44a2846c18f1512b6279690bc\\setup.py'"'"'; __file__='"'"'C:\\Windows\\Temp\\pip-install-5z_34s41\\fasttext_bcbff0f44a2846c18f1512b6279690bc\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Windows\Temp\pip-wheel-gaaogv1s'
       cwd: C:\Windows\Temp\pip-install-5z_34s41\fasttext_bcbff0f44a2846c18f1512b6279690bc\
  Complete output (20 lines):
  C:\ProgramData\Anaconda3\lib\site-packages\setuptools\dist.py:757: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the undersc

In [3]:
data=pd.read_pickle('final-2.pkl')

In [4]:
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,...,product_category_name_english,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,payment_sequential,payment_type,payment_installments,payment_value,review_score_new
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,housewares,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,1,credit_card,1,18.12,1
3,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-18 14:44:43,2017-08-28,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,housewares,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP,1,credit_card,3,37.77,1
4,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-07 18:30:01,2017-08-15,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,housewares,ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP,1,credit_card,1,37.77,1
5,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-11-07 18:04:59,2017-11-13,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,...,housewares,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,1,boleto,1,44.09,1
6,5f49f31e537f8f1a496454b48edbe34d,a7260a6ccba78544ccfaf43f920b7240,delivered,2017-08-24 11:31:28,2017-08-24 11:45:25,2017-08-28 20:12:20,2017-09-14,2,be03d93320192443b8fa24c0ca6ead98,3504c0cb71d7fa48d967e0e4c94d59d9,...,housewares,7a1de9bde89aedca8c5fbad489c5571c,1315,sao paulo,SP,1,credit_card,1,127.45,0


In [5]:
data.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'product_category_name_english',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_score_new'],
      dtype='object')

In [6]:
# Preprocessing the Review text

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [8]:
stopwords_pt = stopwords.words('portuguese')

In [9]:
stopwords_pt.remove('não')
stopwords_pt.remove('nem')

In [10]:
data['review_comment_message'].head()

0    Não testei o produto ainda, mas ele veio corre...
3    Deveriam embalar melhor o produto. A caixa vei...
4    Só achei ela pequena pra seis xícaras ,mais é ...
5                                                  NaN
6    Prezados que porcaria de atendimento ao client...
Name: review_comment_message, dtype: object

In [11]:
data['review_comment_message'].fillna('nao_reveja',inplace=True)

In [12]:
def preprocess_text(texts):
    hyperlinks = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' # check for hyperlinks
    dates = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)\d{2,4}' # check for dates
    currency_symbols = '[R]{0,1}\$[ ]{0,}\d+(,|\.)\d+' # check for currency symbols
    preprocessed_text = []
    stemmer = RSLPStemmer() # portugese nltk stemmer
    for sent in tqdm(texts):
        sent = re.sub(r"[\n\t\-\\\/]"," ",sent)#removing the new line,tab  
        sent = re.sub(hyperlinks, ' url ', sent) # replacing hyperlinks with 'url'
        sent = re.sub(dates, ' ', sent) # removing dates
        sent = re.sub(currency_symbols, ' dinheiro ', sent) # replacing currency symbols with 'dinheiro'
        sent = re.sub('[0-9]+', ' numero ', sent) # removing digits
        sent = re.sub('([nN][ãÃaA][oO]|[ñÑ]| [nN] )', ' negação ', sent) # replacing no with negative
        sent = re.sub('\W', ' ', sent) # removing extra whitespaces
        sent = re.sub('\s+', ' ', sent) # removing extra spaces
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords_pt) # removing stopwords
        sent = ' '.join(stemmer.stem(e.lower()) for e in sent.split()) # stemming the words
        preprocessed_text.append(sent.lower().strip())
        
    return preprocessed_text


In [13]:
processed_text = preprocess_text(data['review_comment_message'].values)

100%|██████████████████████████████████| 96516/96516 [00:40<00:00, 2401.25it/s]


In [14]:
data['review_comment_message']=processed_text

In [15]:
data['review_comment_message'].head()

0    neg test produt aind vei corret boa cond apen ...
3    dev embal melhor produt caix vei tod amass vou...
4                   ach pequen pra seil xíc bom produt
5                                           neg _revej
6    prez porc atend client neg conseg fal atend má...
Name: review_comment_message, dtype: object

In [16]:
col= ['order_id',
 'customer_id',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
  'customer_unique_id',
 'order_item_id',
 'product_id',
 'seller_id',
 'shipping_limit_date',
 'customer_city','customer_zip_code_prefix','product_category_name']

In [17]:
data.drop(columns=col,axis=1,inplace=True)

In [18]:
data.head()

,order_status,price,freight_value,seller_zip_code_prefix,seller_city,seller_state,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,...,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name_english,customer_state,payment_sequential,payment_type,payment_installments,payment_value,review_score_new
0,delivered,29.99,8.72,9350,maua,SP,40.0,268.0,4.0,500.0,...,neg test produt aind vei corret boa cond apen ...,2017-10-11 00:00:00,2017-10-12 03:43:48,housewares,SP,1,credit_card,1,18.12,1
3,delivered,29.99,7.78,9350,maua,SP,40.0,268.0,4.0,500.0,...,dev embal melhor produt caix vei tod amass vou...,2017-08-19 00:00:00,2017-08-20 15:16:36,housewares,SP,1,credit_card,3,37.77,1
4,delivered,29.99,7.78,9350,maua,SP,40.0,268.0,4.0,500.0,...,ach pequen pra seil xíc bom produt,2017-08-08 00:00:00,2017-08-08 23:26:23,housewares,SP,1,credit_card,1,37.77,1
5,delivered,29.99,14.10,9350,maua,SP,40.0,268.0,4.0,500.0,...,neg _revej,2017-11-08 00:00:00,2017-11-10 19:52:38,housewares,SC,1,boleto,1,44.09,1
6,delivered,46.80,67.70,9350,maua,SP,59.0,189.0,3.0,775.0,...,prez porc atend client neg conseg fal atend má...,2017-08-29 00:00:00,2017-08-30 02:26:02,housewares,SP,1,credit_card,1,127.45,0


In [19]:
## split data in to train and spilts

y = data['review_score'].values
x = data.drop(['review_score'],axis=1)

In [20]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=25)

In [21]:
print(X_train.shape , Y_train.shape)

(77212, 25) (77212,)


In [22]:
## Encodeing the cagtegorical featutres
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(binary=True)
count_vec.fit(X_train['order_status'].values)

# Counter-vecctorizer converts text to vector

order_st_train=count_vec.transform(X_train['order_status'].values)
order_st_test= count_vec.transform(X_test['order_status'].values)

name_os=count_vec.get_feature_names()
print(name_os)

['approved', 'canceled', 'delivered', 'invoiced', 'processing', 'shipped', 'unavailable']


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [23]:
# Encoding Categorical _featues :Payment_type

In [24]:
vectorizer=CountVectorizer(binary=True)
vectorizer.fit(X_train['payment_type'].values)

pay_train = vectorizer.transform(X_train['payment_type'].values)
pay_test = vectorizer.transform(X_test['payment_type'].values)

name_py_type = vectorizer.get_feature_names()
print(name_py_type)

['boleto', 'credit_card', 'debit_card', 'voucher']


In [25]:
# Encoding categorical _feature : product category_name_english

In [26]:
vectorizer=CountVectorizer(binary=True)

vectorizer.fit(X_train['product_category_name_english'].values)

pro_train = vectorizer.transform(X_train['product_category_name_english'].values)
pro_test = vectorizer.transform(X_test['product_category_name_english'].values)


pro_type = vectorizer.get_feature_names()
print(pro_type)

['agro_industry_and_commerce', 'air_conditioning', 'art', 'arts_and_craftmanship', 'audio', 'auto', 'baby', 'bed_bath_table', 'books_general_interest', 'books_imported', 'books_technical', 'cds_dvds_musicals', 'christmas_supplies', 'cine_photo', 'computers', 'computers_accessories', 'consoles_games', 'construction_tools_construction', 'construction_tools_lights', 'construction_tools_safety', 'cool_stuff', 'costruction_tools_garden', 'costruction_tools_tools', 'diapers_and_hygiene', 'drinks', 'dvds_blu_ray', 'electronics', 'fashio_female_clothing', 'fashion_bags_accessories', 'fashion_childrens_clothes', 'fashion_male_clothing', 'fashion_shoes', 'fashion_sport', 'fashion_underwear_beach', 'fixed_telephony', 'flowers', 'food', 'food_drink', 'furniture_bedroom', 'furniture_decor', 'furniture_living_room', 'furniture_mattress_and_upholstery', 'garden_tools', 'health_beauty', 'home_appliances', 'home_appliances_2', 'home_comfort_2', 'home_confort', 'home_construction', 'housewares', 'indust

In [27]:
# Encoding categorical_features : customer_state

In [28]:
vectorizer=CountVectorizer(binary=True)

vectorizer.fit(X_train['customer_state'].values)

cus_st_train = vectorizer.transform(X_train['customer_state'].values)
cus_st_test = vectorizer.transform(X_test['customer_state'].values)


cus_st_type = vectorizer.get_feature_names()
print(cus_st_type)

['ac', 'al', 'am', 'ap', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mg', 'ms', 'mt', 'pa', 'pb', 'pe', 'pi', 'pr', 'rj', 'rn', 'ro', 'rr', 'rs', 'sc', 'se', 'sp', 'to']


In [6]:
# Normalizing the numerical features

In [72]:
from sklearn.preprocessing import Normalizer
def normalizer(col):
    normalizer = Normalizer()
    normalizer.fit(X_train[col].values.reshape(-1,1))
    X_train_norm=normalizer.transform(X_train[col].values.reshape(-1,1))
    X_test_norm=normalizer.transform(X_test[col].values.reshape(-1,1))
    
    return X_train_norm , X_test_norm

In [73]:
num_featues=[num for num in data.columns if data[num].dtypes != 'O' and num not in ['review_score','review_score_new'] ]

In [74]:
num_featues

['price',
 'freight_value',
 'seller_zip_code_prefix',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'payment_sequential',
 'payment_installments',
 'payment_value']

In [75]:
train=[]
test=[]
for i in num_featues :
    a,b=normalizer(i)
    train.append(a)
    test.append(b)

In [76]:
from scipy.sparse import hstack , csr_matrix
import numpy as np

X_train_num = np.hstack((train))
X_test_num = np.hstack((test))

print('Final data matrix')
print(X_train_num.shape, Y_train.shape)
print(X_test_num.shape , Y_test.shape)

Final data matrix
(77212, 13) (77212,)
(19304, 13) (19304,)


In [ ]:
# merge